In [1]:
# To install:
!pip install /Users/johnmount/Documents/work/pyvtreat/pkg/dist/vtreat-0.1.tar.gz
#!pip install https://github.com/WinVector/pyvtreat/raw/master/pkg/dist/vtreat-0.1.tar.gz

Processing /Users/johnmount/Documents/work/pyvtreat/pkg/dist/vtreat-0.1.tar.gz
  Stored in directory: /Users/johnmount/Library/Caches/pip/wheels/cf/06/fc/6b2552717486fb6401f19308eec24381555e456e3bd9cfb103
Successfully built vtreat
  Found existing installation: vtreat 0.1
    Uninstalling vtreat-0.1:
      Successfully uninstalled vtreat-0.1


In [2]:
import numpy.random
import pandas
import seaborn
import vtreat # https://github.com/WinVector/pyvtreat


numpy.random.seed(235)
zip = ['z' + str(i+1).zfill(5) for i in range(15)]
d = pandas.DataFrame({'zip':numpy.random.choice(zip, size=1000)})
d["const"] = 1
d["const2"]= "b"
d.head()

,zip,const,const2
0,z00009,1,b
1,z00015,1,b
2,z00002,1,b
3,z00006,1,b
4,z00013,1,b


In [3]:
transform = vtreat.UnsupervisedTreatment(
    params=vtreat.vtreat_parameters({
        'indicator_min_fracton': 0.01,
    }))
transform.params_

{'use_hierarchical_estimate': True,
 'coders': {'clean_copy',
  'deviance_code',
  'impact_code',
  'indicator_code',
  'logit_code',
  'missing_indicator',
  'prevalence_code'},
 'filter_to_recommended': True,
 'indicator_min_fracton': 0.01,
 'cross_validation_plan': <vtreat.KWayCrossPlan at 0x104b2a2b0>,
 'cross_validation_k': 5}

In [4]:
d_treated = transform.fit_transform(d)
d_treated.head()

,zip_prevalence_code,zip_lev_z00009,zip_lev_z00007,zip_lev_z00011,zip_lev_z00013,zip_lev_z00003,zip_lev_z00008,zip_lev_z00015,zip_lev_z00004,zip_lev_z00005,zip_lev_z00001,zip_lev_z00014,zip_lev_z00012,zip_lev_z00006,zip_lev_z00002,zip_lev_z00010
0,0.083,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.064,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0.057,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0.060,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0.071,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
transform.score_frame_

,variable,orig_variable,treatment,y_aware,has_range,PearsonR,significance,recommended,vcount
0,zip_prevalence_code,zip,prevalence_code,False,True,NaN,NaN,True,2.0
1,zip_lev_z00009,zip,indicator_code,False,True,NaN,NaN,True,16.0
2,zip_lev_z00007,zip,indicator_code,False,True,NaN,NaN,True,16.0
3,zip_lev_z00011,zip,indicator_code,False,True,NaN,NaN,True,16.0
4,zip_lev_z00013,zip,indicator_code,False,True,NaN,NaN,True,16.0
5,zip_lev_z00003,zip,indicator_code,False,True,NaN,NaN,True,16.0
6,zip_lev_z00008,zip,indicator_code,False,True,NaN,NaN,True,16.0
7,zip_lev_z00015,zip,indicator_code,False,True,NaN,NaN,True,16.0
8,zip_lev_z00004,zip,indicator_code,False,True,NaN,NaN,True,16.0
9,zip_lev_z00005,zip,indicator_code,False,True,NaN,NaN,True,16.0
